In [1]:
import os
import numpy as np
import shutil
from tqdm import tqdm
import rasterio

In [2]:
fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/train/masks'

In [6]:
len(os.listdir(fp))

5040

In [13]:
im = np.load('../sj_naip_subset.npy')

In [14]:
from PIL import Image
im = Image.fromarray(im)
im.save('../sj_naip_subset.png')

Rename all images in masks/ from .npy to _mask.npy

In [5]:
for filename in os.listdir(f'{fp}masks/'):
    filename1 = filename.replace(".npy", "_mask.npy")
    src =f"{fp}masks/{filename}"  # foldername/filename, if .py file is outside folder
    dst =f"{fp}masks/{filename1}"
    os.rename(src, dst)

Make a train folder

In [8]:
if not os.path.exists(os.path.join(fp, 'train')):
    os.mkdir(os.path.join(fp, 'train'))
#     os.mkdir(os.path.join(fp, 'train', 'raw_tif'))

In [9]:
shutil.move(os.path.join(fp, 'images'), os.path.join(fp, 'train'))
shutil.move(os.path.join(fp, 'masks'), os.path.join(fp, 'train'))
# shutil.move(os.path.join(fp, 'raw_tif'), os.path.join(fp, 'train'))

'/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512/train/masks'

Train-val-test split

In [12]:
def train_val_split(root_dir, save_dir, train_percent):
    if not os.path.exists(save_dir):
        os.mkdir(os.path.join(save_dir))
        os.mkdir(os.path.join(save_dir, 'images'))
        os.mkdir(os.path.join(save_dir, 'masks'))
    
    all_images = list(os.listdir(os.path.join(root_dir, "images")))
    np.random.shuffle(all_images)

    num_train = int(len(all_images) * train_percent)
    val_images = all_images[num_train:]

    for img in tqdm(val_images):
        shutil.move(os.path.join(root_dir, "images", img), os.path.join(save_dir, "images"))
        shutil.move(os.path.join(root_dir, "masks", img.replace(".npy", "_mask.npy")), os.path.join(save_dir, "masks"))

In [13]:
train_val_split(os.path.join(fp, 'train'), os.path.join(fp, 'test'), 0.9)

100%|██████████| 627/627 [00:19<00:00, 32.44it/s]


In [14]:
train_val_split(os.path.join(fp, 'train'), os.path.join(fp, 'val'), 0.9)

100%|██████████| 565/565 [00:18<00:00, 31.02it/s]


copy everything to combined dataset

In [2]:
fp = '/oak/stanford/groups/deho/building_compliance/rgb-footprint-extract/'

In [3]:
if not os.path.exists(fp):
    os.mkdir(fp)
    os.mkdir(os.path.join(fp, 'sj+la'))
    for i in ['train', 'test', 'val']:
        os.mkdir(os.path.join(fp, 'sj+la', i))
        os.mkdir(os.path.join(fp, 'sj+la', i, 'images'))
        os.mkdir(os.path.join(fp, 'sj+la', i, 'masks'))
        os.mkdir(os.path.join(fp, 'sj+la', i, 'masks_wt'))


In [6]:
source_folder = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512/'

In [8]:
for i in ['train', 'test', 'val']:
    for j in ['images', 'masks', 'masks_wt']:
        for fn in os.listdir(os.path.join(source_folder, i, j)):
            shutil.copy(os.path.join(source_folder, i, j, fn), os.path.join(fp, 'sj+la', i, j, fn))

In [ ]:
for file_name in os.listdir(source_folder):
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied', file_name)